In [1]:
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import datetime, duckdb, requests, pyarrow, time, os, random


# Classe criada para captura de historico de dados, armazenamento em banco e previsão de indicadores
class FarmTechMeteoData:

    def __init__(self):
        self.connection = None
        self.scaler = StandardScaler()
        self.model = None
        self.trained_model = False
        self.feature_vars = ['nr_temperatura_ambiente', 'nr_temperatura_solo', 'nr_umidade_ar', 'nr_ph_solo']
        self.target_var = 'ic_irrigacao'
        self.refreshed_data = False


    # Realiza conexão com banco de dados
    def get_db_connection(self):

        # Cria dirtorio para o database
        root_dir = Path.cwd()
        data_dir = root_dir / 'data'
        data_file = data_dir / 'farmtech.db'

        try:
            if not data_dir.exists():
                _ = data_dir.mkdir(parents=True)
            connection = duckdb.connect(data_file)
        except Exception as exc:
            print(str(exc))
            raise
        else:
            # Define conexao de referencia
            self.connection = connection


    def exe_model_training(self, dataframe):
        try:
            # Normalizando as variáveis independentes
            x_scaled = self.scaler.fit_transform(
                dataframe[self.feature_vars]
            )

            # Dividindo os dados em conjunto de treino e teste
            x_train, x_test, y_train, y_test = train_test_split(
                x_scaled, # Dados normalizados para equilibrio de escalas
                dataframe[self.target_var], # dataset com variavel a ser reproduzida na previsao
                test_size= 0.2,  # 20% dos dados para teste e 80% para treino
                random_state= 42 # GPT: Garante que a divisão dos dados seja sempre a mesma, permitindo a reprodutibilidade do processo.
            )

            # Criando o modelo de Random Forest
            self.model = RandomForestClassifier(
                n_estimators= 100, # Quantidade de arvores
                random_state= 42   # Fator de reprodutividade utilizado no treino
            )

            # Treinamento
            self.model.fit(x_train, y_train)
            self.trained_model = True

        except Exception as exc:
            print(str(exc))
            raise


    def get_model_prediction(self, dataframe):
        try:
            # Assegura que o modelo foi treinado previamente
            assert self.trained_model == True, 'O modelo precisa ser treinado'

            dataframe[self.target_var] = self.model.predict(self.scaler.transform(dataframe[self.feature_vars]))

            # Fazendo a previsão com o modelo
            return dataframe

        except Exception as exc:
            print(str(exc))
            raise


    # Função para calcular ph do solo
    def get_ph(
        self,
        temperatura_solo,
        umidade_ar,
        umidade_solo,
        cobertura_nuvem,
        cobertura_nuvem_3km,
        precipitacao
    ):
        """
        O CHAT GPT Disse:
        -
        Temperatura do Solo:
            Quanto mais alta a temperatura do solo, mais ácido ele tende a ser devido à maior atividade microbiana e decomposição orgânica.
            O peso é maior do que a temperatura do ar, pois a temperatura do solo é mais diretamente ligada às propriedades do solo.
        Umidade do Solo:
            Alta umidade do solo tende a neutralizar o pH, evitando que o solo fique muito ácido.
        Umidade Relativa do Ar e Cobertura de Nuvens:
            Indicam condições climáticas que podem afetar indiretamente a retenção de umidade no solo, mas têm um impacto menor no cálculo do pH.
        Cobertura de Nuvens Baixa:
            Pode ter maior impacto local, como manter o solo mais úmido e fresco.
        Precipitação:
            Cada mm de chuva (precipitação) contribui com um aumento proporcional no pH, já que ela dilui a concentração de íons ácidos no solo.
        Limitações:
            O pH é limitado entre 3 e 9 para refletir valores realistas encontrados em solos agrícolas e naturais.
        """

        try:
            # Fatores que influenciam o pH
            fator_temperatura_solo = 7 - (temperatura_solo - 25) * 0.15 # Ajuste para temperatura do solo
            fator_umidade_solo = umidade_solo * 0.05 # Umidade do solo tem maior peso no pH
            fator_umidade_ar = umidade_ar * 0.01 # Umidade do ar tem impacto secundário
            fator_nuvens = (cobertura_nuvem + cobertura_nuvem_3km) * 0.005 # Nuvens têm impacto indireto
            fator_precipitacao = precipitacao * 0.1 # Chuva alta neutraliza a acidez (aumenta pH)

            # Estimativa de pH
            ph_base = fator_temperatura_solo + fator_umidade_solo + fator_umidade_ar + fator_nuvens + fator_precipitacao
            ph_referencia = random.randint(5, 9)

            return round(max(3, min(ph_base, ph_referencia)), 2)
        except Exception as exc:
            print(str(exc))
            raise


    # Função para recuperar dados metereologicos
    def get_weather(
        self,
        latitude= None,
        longitude = None,
        start= None,
        end= None
    ):
        try:
            # Validação de datas de referencia
            date = [*datetime.datetime.now(tz= datetime.timezone.utc).timetuple()]

            if start and end:
                start_date = start
                end_date = end
            else:
                start_date = (datetime.datetime(*date[:4], 0) - datetime.timedelta(days= 90)).date()
                end_date = (datetime.datetime(*date[:4], 0) + datetime.timedelta(days= 7)).date()

            # Validação de latitude e longitude
            if latitude and longitude:
                lat = latitude
                lon = longitude
            else:
                lat = '36.74068642011221'   # PXR7+7JH Kerman, Califórnia, EUA
                lon = '-120.03588642549094' # PXR7+7JH Kerman, Califórnia, EUA

            # Composição de url
            url_absolute = 'https://api.open-meteo.com/v1/forecast{}'
            url_relative = '?timezone={}&latitude={}&longitude={}&start_date={}&end_date={}&hourly={}'
            url_timezone = 'America/Sao_Paulo'
            url_latitude =  lat # PXR5+JXJ Kerman, Califórnia, EUA
            url_longitude = lon # PXR5+JXJ Kerman, Califórnia, EUA
            url_vars = [
                'temperature_2m', # °C (°F) | Temperatura ambiente (2m do solo)
                'relative_humidity_2m', # % | Umidade relativa do ar (2m do solo)
                'precipitation', # mm (inch) | Precipitação total
                'rain', # mm (inch) | Volume de chuva
                'soil_temperature_54cm', # °C (°F) | Temperatura do solo
                'soil_moisture_9_to_27cm', # m³/m³ | Umidade média do solo
                'cloud_cover', # % | Cobertura de nuvens total
                'cloud_cover_low' # % | Cobertura de nuvens a 3km de altitude
            ]

            # Recupera dados de tempo dos ultimos 90 dias
            response = requests.get(
                url = url_absolute.format(
                    url_relative.format(
                        url_timezone,
                        url_latitude,
                        url_longitude,
                        start_date.isoformat(),
                        end_date.isoformat(),
                        ','.join(url_vars)
                    )
                ),
                headers= {
                    'Content-Type': 'application/json'
                }
            )

            assert response.status_code == 200, \
                f'Error getting data. status_code= {response.status_code} | message= {response.text}'

            data = response.json()
            tempo = data.get('hourly', {}).get('time')
            temperatura = data.get('hourly', {}).get('temperature_2m')
            temperatura_solo = data.get('hourly', {}).get('soil_temperature_54cm')
            umidade_ar = data.get('hourly', {}).get('relative_humidity_2m')
            umidade_solo = data.get('hourly', {}).get('soil_moisture_9_to_27cm')
            cobertura_nuvem = data.get('hourly', {}).get('cloud_cover')
            cobertura_nuvem_3km = data.get('hourly', {}).get('cloud_cover_low')
            chuva = data.get('hourly', {}).get('rain')
            precipitacao = data.get('hourly', {}).get('precipitation')

            return [
                {
                    'id_tempo': int(time.mktime(datetime.datetime.strptime(tempo[t], '%Y-%m-%dT%H:%M').timetuple())),
                    'ds_localidade': 'Small Ville',
                    'nr_latitude': response.json().get('latitude'),
                    'nr_longitude': response.json().get('longitude'),
                    'dh_indicador': datetime.datetime.strptime(tempo[t], '%Y-%m-%dT%H:%M'),
                    'dt_indicador': datetime.datetime.strptime(tempo[t], '%Y-%m-%dT%H:%M').date(),
                    'hr_indicador': datetime.datetime.strptime(tempo[t], '%Y-%m-%dT%H:%M').time(),
                    "nr_elevacao": response.json().get('elevation'),
                    'nr_pct_nuvens': cobertura_nuvem[t],
                    'nr_pct_nuvens_baixas': cobertura_nuvem_3km[t],
                    'nr_volume_chuva': chuva[t],
                    'nr_precipitacao': precipitacao[t],
                    'nr_temperatura_ambiente': temperatura[t],
                    'nr_temperatura_solo': temperatura_solo[t],
                    'nr_umidade_ar': umidade_ar[t],
                    'nr_umidade_solo': umidade_solo[t],
                    'nr_ph_solo': self.get_ph(
                        temperatura_solo= temperatura_solo[t],
                        umidade_ar= umidade_ar[t],
                        umidade_solo= umidade_solo[t],
                        cobertura_nuvem= cobertura_nuvem[t],
                        cobertura_nuvem_3km= cobertura_nuvem_3km[t],
                        precipitacao= precipitacao[t]
                    )
                } \
                for t in range(0, len(tempo) , 1) \
                    if None not in [
                        temperatura[t],
                        temperatura_solo[t],
                        umidade_ar[t],
                        umidade_solo[t],
                        cobertura_nuvem[t],
                        cobertura_nuvem_3km[t],
                        precipitacao[t]
                    ]
            ]

        except Exception as exc:
            print(str(exc))
            raise


    # Função para atualizar dadbase com dados metereologicos
    def exe_refresh_meteo_data(self):
        try:
            # Estabelece conexão com database, caso nao exista
            if not self.connection:
                self.get_db_connection()

            # Verifica cria tabela, caso nao exista
            self.connection.execute(
                """
                create table if not exists t_meteo_historic
                (
                    id_tempo integer PRIMARY KEY,
                    ds_localidade varchar,
                    nr_latitude double,
                    nr_longitude double,
                    dh_indicador timestamp,
                    dt_indicador date,
                    hr_indicador time,
                    nr_elevacao double,
                    nr_pct_nuvens double,
                    nr_pct_nuvens_baixas double,
                    nr_volume_chuva double,
                    nr_precipitacao double,
                    nr_temperatura_ambiente double,
                    nr_temperatura_solo double,
                    nr_umidade_ar double,
                    nr_umidade_solo double,
                    nr_ph_solo double
                )
                """
            )

            print('t_meteo_historic criada')

            # Constroi arrow table baseado no resultado da API
            t_meteo_temp = pyarrow.Table.from_pylist(mapping= self.get_weather())

            # Insere dados metereologicos recuperados de API
            self.connection.execute(
                """
                insert into
                    t_meteo_historic
                (
                    id_tempo,
                    ds_localidade,
                    nr_latitude,
                    nr_longitude,
                    dh_indicador,
                    dt_indicador,
                    hr_indicador,
                    nr_elevacao,
                    nr_pct_nuvens,
                    nr_pct_nuvens_baixas,
                    nr_volume_chuva,
                    nr_precipitacao,
                    nr_temperatura_ambiente,
                    nr_temperatura_solo,
                    nr_umidade_ar,
                    nr_umidade_solo,
                    nr_ph_solo
                )
                select
                    x.id_tempo,
                    x.ds_localidade,
                    x.nr_latitude,
                    x.nr_longitude,
                    x.dh_indicador,
                    x.dt_indicador,
                    x.hr_indicador,
                    x.nr_elevacao,
                    x.nr_pct_nuvens,
                    x.nr_pct_nuvens_baixas,
                    x.nr_volume_chuva,
                    x.nr_precipitacao,
                    x.nr_temperatura_ambiente,
                    x.nr_temperatura_solo,
                    x.nr_umidade_ar,
                    x.nr_umidade_solo,
                    x.nr_ph_solo
                from
                    t_meteo_temp as x
                left join
                    t_meteo_historic as y
                on
                    x.id_tempo = y.id_tempo
                where
                    y.id_tempo is null
                """
            )

            self.refreshed_data = True

        except Exception as exc:
            print(str(exc))
            raise


    # Recupera dados metereologicos dos ultimos 90 dias do banco e retorna dataframe pandas
    def get_meteo_training_data(self):
        try:
            # Estabelece conexão com database, caso nao exista
            if not self.connection:
                self.get_db_connection()

            # Garante atualizacao de dados para uso
            if not self.refreshed_data:
                self.exe_refresh_meteo_data()

            # Apaga tabela com dados defasados, caso exista
            self.connection.execute(
                """
                drop table if exists t_meteo_trained
                """
            )

            # Cria tabela de treino com dados mais recentes
            self.connection.execute(
                """
                create table t_meteo_trained as
                select
                    x.id_tempo,
                    x.nr_temperatura_ambiente,
                    x.nr_temperatura_solo,
                    x.nr_umidade_ar,
                    x.nr_ph_solo
                from
                    t_meteo_historic as x
                where
                    x.dt_indicador >= current_date - INTERVAL 90 DAY
                and
                    x.dt_indicador <  current_date
                """
            )

            print('t_meteo_trained criada')

            df = self.connection.sql('select * from t_meteo_trained').df()

            df[self.target_var] = df.apply(lambda _: \
                1 if (_['nr_umidade_ar'] > 60.0) and (_['nr_ph_solo'] >= 6.0 and _['nr_ph_solo'] <= 8.0) else 0, \
                axis=1)

            # Retorna dataframe com base para testes
            return df

        except Exception as exc:
            print(str(exc))
            raise


    # Recupera dados metereologicos dos ultimos 90 dias do banco e retorna dataframe pandas
    def get_meteo_prediction_data(self):
        try:
            # Estabelece conexão com database, caso nao exista
            if not self.connection:
                self.get_db_connection()

            # Garante atualizacao de dados para uso
            if not self.refreshed_data:
                self.exe_refresh_meteo_data()

            if not self.trained_model:
                self.exe_model_training(dataframe= self.get_meteo_training_data())

            # Apaga tabela com dados defasados, caso exista
            self.connection.execute(
                """
                drop table if exists t_meteo_predicted
                """
            )

            # Recupera dataframe para previsao de irrigação
            df = self.connection.sql(
                """
                select
                    x.id_tempo,
                    x.ds_localidade,
                    x.nr_latitude,
                    x.nr_longitude,
                    x.dh_indicador,
                    x.dt_indicador,
                    x.hr_indicador,
                    x.nr_elevacao,
                    x.nr_pct_nuvens,
                    x.nr_pct_nuvens_baixas,
                    x.nr_volume_chuva,
                    x.nr_precipitacao,
                    x.nr_temperatura_ambiente,
                    x.nr_temperatura_solo,
                    x.nr_umidade_ar,
                    x.nr_umidade_solo,
                    x.nr_ph_solo
                from
                    t_meteo_historic as x
                where
                    x.dt_indicador >= current_date - INTERVAL 90 DAY
                """
            )\
            .df()

            # Executa previsao
            df_predicted = self.get_model_prediction(dataframe= df)

            # Grava dataframe de previsao no banco
            self.connection.execute(
                """
                create table t_meteo_predicted as select * from df_predicted
                """
            )

            print('t_meteo_predicted criada')

            return df_predicted

        except Exception as exc:
            print(str(exc))
            raise

In [2]:
from pathlib import Path
from queue import Queue
from paho.mqtt import client as mqtt
import datetime, duckdb, json, time


class FarmTechSensorData:

    # Metodo padrão de instancia
    def __init__(self, broker = 'broker.mqtt-dashboard.com', topic = 'farmTechSolutions', port= 1883):
        self.connection = None
        self.broker = broker
        self.topic = topic
        self.port = port
        self.client = None
        self.messages = Queue()


    # Realiza conexão com banco de dados
    def get_db_connection(self):
        # Cria dirtorio para o database
        root_dir = Path.cwd()
        data_dir = root_dir / 'data'
        data_file = data_dir / 'farmtech.db'

        try:
            assert data_file.exists() == True, \
                'Database nao existe ou não foi encontrado'
            connection = duckdb.connect(data_file)
        except Exception as exc:
            print(str(exc))
            raise
        else:
            # Define conexao de referencia
            self.connection = connection


    # Callback executado quando o cliente se conecta ao broker
    def on_connect(self, client, userdata, flags, rc):
        if rc == 0:
            client.subscribe(self.topic)
            print('Conexão realizada com sucesso.')
        else:
            print(f'Falha na conexão, código de retorno: {rc}.')


    # Callback executado quando a conexão é perdida
    def on_disconnect(self, client, userdata, rc):
        if rc != 0:
            print('Conexão com o broker perdida. Tentando reconectar...')
        else:
            print('Desconectado do broker com sucesso.')


    # Callback executado ao receber uma mensagem no tópico inscrito e gravar no banco de dados
    def on_message(self, client, userdata, msg):
        try:
            # Estabelece conexão com database
            if not self.connection:
                self.get_db_connection()

            message = json.loads(msg.payload.decode('utf-8'))

            sensor_date = [*datetime.datetime.strptime(message.get('data'), '%d/%m/%Y').timetuple()]
            sensor_time = [int(t) for t in message.get('hora').split(':')]
            sensor_datetime = datetime.datetime(*sensor_date[:3], *sensor_time[:3])
            meteo_datetime = datetime.datetime(*sensor_date[:3], *sensor_time[:1])

            # Define registro a ser incluida no banco
            sensor_data = {
                'id_sensor': int(time.mktime(sensor_datetime.timetuple())),
                'id_tempo': int(time.mktime(meteo_datetime.timetuple())),
                'dh_sensor': sensor_datetime,
                'dt_sensor': sensor_datetime.date(),
                'hr_sensor': sensor_datetime.time(),
                'nr_umidade_ar': message.get('humidade'),
                'nr_ph_solo': message.get('ph'),
                'ic_fosforo': message.get('fosforo'),
                'ic_potassio': message.get('potassio'),
                'ic_irrigacao': message.get('irrigacao')
            }

            self.connection.execute(
                """
                create table if not exists t_sensor_historic
                (
                    id_sensor integer PRIMARY KEY,
                    id_tempo integer,
                    dh_sensor timestamp,
                    dt_sensor date,
                    hr_sensor time,
                    nr_umidade_ar double,
                    nr_ph_solo double,
                    ic_fosforo integer,
                    ic_potassio integer,
                    ic_irrigacao integer
                )
                """
            )

            print('t_sensor_historic criada')

            sensor_query = \
                """
                insert into t_sensor_historic
                (
                    id_sensor,
                    id_tempo,
                    dh_sensor,
                    dt_sensor,
                    hr_sensor,
                    nr_umidade_ar,
                    nr_ph_solo,
                    ic_fosforo,
                    ic_potassio,
                    ic_irrigacao
                )
                values
                (
                    ?,?,?,?,?,?,?,?,?,?
                )
                on conflict do nothing
                """

            self.connection.execute(
                sensor_query, (
                    sensor_data.get('id_sensor'),
                    sensor_data.get('id_tempo'),
                    sensor_data.get('dh_sensor'),
                    sensor_data.get('dt_sensor'),
                    sensor_data.get('hr_sensor'),
                    sensor_data.get('nr_umidade_ar'),
                    sensor_data.get('nr_ph_solo'),
                    sensor_data.get('ic_fosforo'),
                    sensor_data.get('ic_potassio'),
                    sensor_data.get('ic_irrigacao')
                )
            )

            # Exibe os dados recebidos em formato JSON
            self.messages.put(item= message)

        except json.JSONDecodeError as exc:
            print(f'Erro ao decodificar a mensagem JSON. {str(exc)}')
            raise


    # Recupera client baseado em parametros de instancia
    def get_session(self):
        try:
            self.client = mqtt.Client()  
            self.client.on_connect = self.on_connect
            self.client.on_message = self.on_message
            self.client.on_disconnect = self.on_disconnect
            self.client.connect(self.broker, self.port)
            self.client.loop_start()
        except Exception as exc:
            print(f'Problemas ao adquirir client {str(exc)}')
            raise


    # Recupera mensagem enviada para o tópico do broker
    def get_messages(self):
        while True:
            yield self.messages.get(block= True)


    # Métodos para inicializar o context manager
    def __enter__(self):
        if not self.client:
            self.get_session()
        return self


    # Métodos para finalizar o context manager
    def __exit__(self, exc_type, exc_val, exc_tb):
        if self.client:
            self.client.loop_stop()

Conexao com banco

In [3]:
def get_db_connection():
    root_dir = Path.cwd()
    data_dir = root_dir / 'data'
    data_file = data_dir / 'farmtech.db'

    try:
        if not data_dir.exists():
            _ = data_dir.mkdir(parents=True)
        connection = duckdb.connect(data_file)
    except Exception as exc:
        print(str(exc))
        raise
    else:
        return connection

connection = get_db_connection()

Teste Modelo

In [4]:
meteo = FarmTechMeteoData()

In [5]:
data = meteo.get_meteo_prediction_data()
data.head()

t_meteo_historic criada
t_meteo_trained criada
t_meteo_predicted criada


,id_tempo,ds_localidade,nr_latitude,nr_longitude,dh_indicador,dt_indicador,hr_indicador,nr_elevacao,nr_pct_nuvens,nr_pct_nuvens_baixas,nr_volume_chuva,nr_precipitacao,nr_temperatura_ambiente,nr_temperatura_solo,nr_umidade_ar,nr_umidade_solo,nr_ph_solo,ic_irrigacao
0,1725955200,Small Ville,36.72757,-120.03746,2024-09-10 08:00:00,2024-09-10,08:00:00,72.0,0.0,0.0,0.0,0.0,17.4,28.0,65.0,0.113,7.00,1
1,1725958800,Small Ville,36.72757,-120.03746,2024-09-10 09:00:00,2024-09-10,09:00:00,72.0,0.0,0.0,0.0,0.0,16.9,28.0,70.0,0.113,7.26,1
2,1725962400,Small Ville,36.72757,-120.03746,2024-09-10 10:00:00,2024-09-10,10:00:00,72.0,0.0,0.0,0.0,0.0,16.3,28.0,73.0,0.113,6.00,1
3,1725966000,Small Ville,36.72757,-120.03746,2024-09-10 11:00:00,2024-09-10,11:00:00,72.0,0.0,0.0,0.0,0.0,16.2,28.0,74.0,0.113,7.00,1
4,1725969600,Small Ville,36.72757,-120.03746,2024-09-10 12:00:00,2024-09-10,12:00:00,72.0,6.0,0.0,0.0,0.0,19.5,28.0,60.0,0.113,7.19,0


In [6]:
connection.sql('select * from t_meteo_historic limit 10')

┌────────────┬───────────────┬─────────────┬──────────────┬─────────────────────┬──────────────┬──────────────┬─────────────┬───────────────┬──────────────────────┬─────────────────┬─────────────────┬─────────────────────────┬─────────────────────┬───────────────┬─────────────────┬────────────┐
│  id_tempo  │ ds_localidade │ nr_latitude │ nr_longitude │    dh_indicador     │ dt_indicador │ hr_indicador │ nr_elevacao │ nr_pct_nuvens │ nr_pct_nuvens_baixas │ nr_volume_chuva │ nr_precipitacao │ nr_temperatura_ambiente │ nr_temperatura_solo │ nr_umidade_ar │ nr_umidade_solo │ nr_ph_solo │
│   int32    │    varchar    │   double    │    double    │      timestamp      │     date     │     time     │   double    │    double     │        double        │     double      │     double      │         double          │       double        │    double     │     double      │   double   │
├────────────┼───────────────┼─────────────┼──────────────┼─────────────────────┼──────────────┼──────────────┼─

In [7]:
connection.sql('select * from t_meteo_trained limit 10')

┌────────────┬─────────────────────────┬─────────────────────┬───────────────┬────────────┐
│  id_tempo  │ nr_temperatura_ambiente │ nr_temperatura_solo │ nr_umidade_ar │ nr_ph_solo │
│   int32    │         double          │       double        │    double     │   double   │
├────────────┼─────────────────────────┼─────────────────────┼───────────────┼────────────┤
│ 1725955200 │                    17.4 │                28.0 │          65.0 │        7.0 │
│ 1725958800 │                    16.9 │                28.0 │          70.0 │       7.26 │
│ 1725962400 │                    16.3 │                28.0 │          73.0 │        6.0 │
│ 1725966000 │                    16.2 │                28.0 │          74.0 │        7.0 │
│ 1725969600 │                    19.5 │                28.0 │          60.0 │       7.19 │
│ 1725973200 │                    23.4 │                27.9 │          47.0 │        6.0 │
│ 1725976800 │                    26.9 │                27.9 │          37.0 │  

In [8]:
connection.sql('select * from t_meteo_predicted limit 10')

┌────────────┬───────────────┬─────────────┬──────────────┬─────────────────────┬─────────────────────┬──────────────┬─────────────┬───────────────┬──────────────────────┬─────────────────┬─────────────────┬─────────────────────────┬─────────────────────┬───────────────┬─────────────────┬────────────┬──────────────┐
│  id_tempo  │ ds_localidade │ nr_latitude │ nr_longitude │    dh_indicador     │    dt_indicador     │ hr_indicador │ nr_elevacao │ nr_pct_nuvens │ nr_pct_nuvens_baixas │ nr_volume_chuva │ nr_precipitacao │ nr_temperatura_ambiente │ nr_temperatura_solo │ nr_umidade_ar │ nr_umidade_solo │ nr_ph_solo │ ic_irrigacao │
│   int32    │    varchar    │   double    │    double    │      timestamp      │      timestamp      │     time     │   double    │    double     │        double        │     double      │     double      │         double          │       double        │    double     │     double      │   double   │    int64     │
├────────────┼───────────────┼─────────────┼──

Teste MQTT

In [9]:
counter = 0

with FarmTechSensorData() as session:
    for message in session.get_messages():
        print(message)

        if counter == 10:
            break
        else:
            counter += 1

/tmp/ipykernel_31650/4112097132.py:151: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  self.client = mqtt.Client()


Conexão realizada com sucesso.
t_sensor_historic criada
{'humidade': 47, 'ph': 3.4, 'fosforo': '0', 'potassio': '0', 'data': '09/12/2024', 'hora': '12:08:22', 'irrigacao': 0}
t_sensor_historic criada
{'humidade': 47, 'ph': 3.4, 'fosforo': '0', 'potassio': '0', 'data': '09/12/2024', 'hora': '12:08:23', 'irrigacao': 0}
t_sensor_historic criada
{'humidade': 47, 'ph': 3.4, 'fosforo': '0', 'potassio': '0', 'data': '09/12/2024', 'hora': '12:08:24', 'irrigacao': 0}
t_sensor_historic criada
{'humidade': 47, 'ph': 3.4, 'fosforo': '0', 'potassio': '0', 'data': '09/12/2024', 'hora': '12:08:26', 'irrigacao': 0}
t_sensor_historic criada
{'humidade': 47, 'ph': 3.4, 'fosforo': '0', 'potassio': '0', 'data': '09/12/2024', 'hora': '12:08:27', 'irrigacao': 0}
t_sensor_historic criada
{'humidade': 47, 'ph': 3.4, 'fosforo': '0', 'potassio': '0', 'data': '09/12/2024', 'hora': '12:08:29', 'irrigacao': 0}
t_sensor_historic criada
{'humidade': 47, 'ph': 3.4, 'fosforo': '0', 'potassio': '0', 'data': '09/12/2024

In [10]:
connection.sql('select * from t_sensor_historic limit 10')

┌────────────┬────────────┬─────────────────────┬────────────┬───────────┬───────────────┬────────────┬────────────┬─────────────┬──────────────┐
│ id_sensor  │  id_tempo  │      dh_sensor      │ dt_sensor  │ hr_sensor │ nr_umidade_ar │ nr_ph_solo │ ic_fosforo │ ic_potassio │ ic_irrigacao │
│   int32    │   int32    │      timestamp      │    date    │   time    │    double     │   double   │   int32    │    int32    │    int32     │
├────────────┼────────────┼─────────────────────┼────────────┼───────────┼───────────────┼────────────┼────────────┼─────────────┼──────────────┤
│ 1733746102 │ 1733745600 │ 2024-12-09 12:08:22 │ 2024-12-09 │ 12:08:22  │          47.0 │        3.4 │          0 │           0 │            0 │
│ 1733746103 │ 1733745600 │ 2024-12-09 12:08:23 │ 2024-12-09 │ 12:08:23  │          47.0 │        3.4 │          0 │           0 │            0 │
│ 1733746104 │ 1733745600 │ 2024-12-09 12:08:24 │ 2024-12-09 │ 12:08:24  │          47.0 │        3.4 │          0 │        